In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [2]:
f = pd.read_csv('final raw data.csv', na_filter=False)

In [4]:
data = f.drop(['Full count Pupae', 'Household_ID', 'Full count Larvae'], axis=1)

In [5]:
data.rename(columns={'Larvae': 'class'}, inplace=True)
data.head(5)

,Hamlet,Container_ID,Container_position,Water_level,Lid,Fish,Meso,Micronecta,Source,Treatment,class
0,0,1,0,25,1,0,0,0,0,0,1
1,0,2,1,25,1,0,0,0,1,0,0
2,0,5,0,25,0,0,0,0,1,0,0
3,0,6,0,25,1,0,0,0,0,1,0
4,0,5,1,30,1,0,0,0,0,1,0


In [7]:
pos_data = data[data['class'] >= 1]
neg_data = data[data['class'] == 0]

print (pos_data.shape)
print (neg_data.shape)

(50, 11)
(500, 11)


In [86]:
pos_len = len(pos_data.index)
random_pos_data = pos_data.take(np.random.permutation(len(pos_data))[:pos_len])

samples = int(0.75 * pos_len)
train_pos_data = random_pos_data[:samples]
test_pos_data = random_pos_data[samples:]


In [87]:
final_pos_data = pd.concat([train_pos_data]*5)
times = len(final_pos_data)
final_neg_data = neg_data.take(np.random.permutation(len(neg_data))[:times])
frames = [final_pos_data, final_neg_data]
final_data = pd.concat(frames)
print final_data.shape

(370, 11)


In [88]:
features = ['Hamlet', 'Container_ID', 'Water_level', 'Container_position', 'Meso', 'Treatment']
Y = final_data['class']
X = final_data[features]


In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [90]:
classifiers = {'Logistic': LogisticRegression(),
               'GaussianNB':GaussianNB(),
               'BernoulliNB': BernoulliNB(),
               'KNeighbors': KNeighborsClassifier(),
               'RandomForest': RandomForestClassifier(),
               'Decision': DecisionTreeClassifier()}

In [91]:
from sklearn.cross_validation import train_test_split
test_pos_data_X = test_pos_data[features]
test_pos_data_y = test_pos_data['class']

feature_cols = ['BernoulliNB', 'RandomForest', 'GaussianNB', 'Logistic', 'Decision', 'KNeighbors']
precision_score =[]

for name, clf in classifiers.items():
    clf.fit(X,Y)
    score = clf.predict(test_pos_data_X)
    correct_pred = np.count_nonzero(score == 1) 
    precision = round(float(correct_pred)/13,2)
    #print name + " had a precision score of " + str(precision)
    precision_score.append(precision)

pd.DataFrame({'feature':feature_cols, 'precision':precision_score})

    

,feature,precision
0,BernoulliNB,0.54
1,RandomForest,0.23
2,GaussianNB,0.69
3,Logistic,0.54
4,Decision,0.38
5,KNeighbors,0.62


In [92]:
from sklearn.preprocessing import MinMaxScaler

X_scaled = MinMaxScaler().fit_transform(X)

from sklearn.decomposition import PCA
X_pca = PCA(n_components=2).fit_transform(X_scaled)

print (X_pca.shape)

(370, 2)


In [93]:
from sklearn import svm
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score

pca = PCA(n_components=0.8,whiten=True)
train_x = pca.fit_transform(X)
test_x = pca.transform(test_pos_data_X)
train_y = Y

svc = svm.SVC(kernel='rbf',C=2)
svc.fit(train_x, train_y)

y_pred = svc.predict(test_x)
precision = round(float(correct_pred)/13,2)
print precision

0.62
